In [3]:
import mysql.connector
import pandas as pd

connection = mysql.connector.connect(
    host= 'localhost',         
    database = 'AIO2025_2',
    user= 'root',             
    password= '123456'       
)
cursor = connection.cursor()

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
cursor.execute("""
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName VARCHAR(100) NOT NULL,
    Category VARCHAR(50) NOT NULL,
    Price DECIMAL(10, 2) NOT NULL, 
    StockQuantity INTEGER NOT NULL
);
""")

In [6]:
cursor.execute("""
INSERT INTO Products (ProductID, ProductName, Category, Price, StockQuantity) VALUES
(1, 'Laptop Dell XPS', 'Laptop', '25000000', '10'),
(2, 'iPhone 14', 'Điện thoại', '22000000', '15'),
(3, 'Tai nghe Sony', 'Phụ kiện', '3000000', '20'),
(4, 'Bàn phím Cơ', 'Phụ kiện', '1500000', '50'),
(5, 'MacBook Air M2', 'Laptop', '28000000', '8');
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE Customers (
    CustomerID INTEGER PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    PhoneNumber VARCHAR(15) UNIQUE,
    Email VARCHAR(100) UNIQUE
);
""")

In [8]:
cursor.execute("""
INSERT INTO Customers (CustomerID, FullName, PhoneNumber, Email) VALUES
(1, 'Nguyễn Văn A', '0987654321', 'a@example.com'),
(2, 'Trần Thị B', '0976543210', 'b@example.com'),
(3, 'Lê Văn C', '0965432109', 'c@example.com'),
(4, 'Phạm Thị D', '0945678123', 'd@example.com'),
(5, 'Bùi Văn E', '0936789123', 'e@example.com');
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE Salespersons (
    SalespersonID INTEGER PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    PhoneNumber VARCHAR(15) UNIQUE
);
""")

In [11]:
cursor.execute("""
INSERT INTO Salespersons (SalespersonID, FullName, PhoneNumber) VALUES
(1, 'Lê Hoàng Nam', '0934567890'),
(2, 'Trần Quốc Bảo', '0945678123'),
(3, 'Nguyễn Minh Hà', '0956789123');
""")

connection.commit()

In [14]:
cursor.execute("""
CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER ,
    SalespersonID INTEGER, 
    OrderDate DATE, 
    TotalAmount DECIMAL(10, 2),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (SalespersonID) REFERENCES Salespersons(SalespersonID)
);
""")

In [15]:
cursor.execute("""
INSERT INTO Orders (OrderID, CustomerID, SalespersonID, OrderDate, TotalAmount) VALUES
(1, 1, 1, '2024-03-10', '25000000'),
(2, 2, 2, '2024-03-11', '3000000'),
(3, 3, 3, '2024-03-12', '22000000');
""")

connection.commit()

In [16]:
cursor.execute("""
CREATE TABLE OrderDetails (
    OrderDetailID INTEGER PRIMARY KEY,
    OrderID INTEGER,
    ProductID INTEGER,
    Quantity INTEGER, 
    Subtotal DECIMAL(10, 2), 
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
""")

In [17]:
cursor.execute("""
INSERT INTO OrderDetails (OrderDetailID, OrderID, ProductID, Quantity, Subtotal) VALUES
(1, 1, 1, 1, '25000000'),
(2, 2, 3, 1, '3000000'),
(3, 3, 2, 1, '22000000');
""")

connection.commit()

In [19]:
query = ("""
SELECT * FROM Products;
""")

data_3_1 = pd.read_sql_query(query, connection)
print(data_3_1)

   ProductID      ProductName    Category       Price  StockQuantity
0          1  Laptop Dell XPS      Laptop  25000000.0             10
1          2        iPhone 14  Điện thoại  22000000.0             15
2          3    Tai nghe Sony    Phụ kiện   3000000.0             20
3          4      Bàn phím Cơ    Phụ kiện   1500000.0             50
4          5   MacBook Air M2      Laptop  28000000.0              8


In [20]:
query = ("""
SELECT * FROM Orders
WHERE CustomerID = 1;
""")

data_3_2 = pd.read_sql_query(query, connection)
print(data_3_2)

   OrderID  CustomerID  SalespersonID   OrderDate  TotalAmount
0        1           1              1  2024-03-10   25000000.0


In [21]:
query = ("""
SELECT SalespersonID, SUM(TotalAmount) AS TotalSales
 FROM Orders
GROUP BY SalespersonID;
""")

data_3_3 = pd.read_sql_query(query, connection)
print(data_3_3)

   SalespersonID  TotalSales
0              1  25000000.0
1              2   3000000.0
2              3  22000000.0


In [27]:
query = ("""
SELECT b.FullName, SUM(a.TotalAmount) AS TotalSales
 FROM Orders a
LEFT JOIN Salespersons b ON a.SalespersonID = b.SalespersonID
GROUP BY b.FullName
ORDER BY TotalSales DESC
LIMIT 1;
""")

data_4_1 = pd.read_sql_query(query, connection)
print(data_4_1)

       FullName  TotalSales
0  Lê Hoàng Nam  25000000.0


In [26]:
query = ("""
SELECT b.ProductName, SUM(a.Subtotal) AS TotalSales
 FROM OrderDetails a
LEFT JOIN Products b ON a.ProductID = b.ProductID
GROUP BY b.ProductName
ORDER BY TotalSales DESC
LIMIT 1;
""")

data_4_2 = pd.read_sql_query(query, connection)
print(data_4_2)

       ProductName  TotalSales
0  Laptop Dell XPS  25000000.0


In [29]:
query = ("""
SELECT b.FullName, SUM(a.TotalAmount) AS TotalSales
 FROM Orders a
LEFT JOIN Customers b ON a.CustomerID = b.CustomerID
GROUP BY b.FullName
ORDER BY TotalSales DESC
LIMIT 3;
""")

data_4_3 = pd.read_sql_query(query, connection)
print(data_4_3)

       FullName  TotalSales
0  Nguyễn Văn A  25000000.0
1      Lê Văn C  22000000.0
2    Trần Thị B   3000000.0


In [30]:
query = ("""
SELECT MONTH(OrderDate) AS Month, SUM(TotalAmount) AS TotalSales
 FROM Orders
GROUP BY MONTH(OrderDate)
ORDER BY TotalSales DESC;
""")

data_4_4 = pd.read_sql_query(query, connection)
print(data_4_4)

   Month  TotalSales
0      3  50000000.0


In [32]:
query = ("""
SELECT b.ProductName, SUM(a.Quantity) AS TotalQuantity
 FROM OrderDetails a
LEFT JOIN Products b ON a.ProductID = b.ProductID
GROUP BY b.ProductName
ORDER BY TotalQuantity DESC;
""")

data_4_5 = pd.read_sql_query(query, connection)
print(data_4_5)

       ProductName  TotalQuantity
0  Laptop Dell XPS            1.0
1    Tai nghe Sony            1.0
2        iPhone 14            1.0
